<a href="https://colab.research.google.com/github/ag4267research1/Solving-Poisson-s-Equation-with-GNO/blob/main/GNOPDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U neuraloperator
!pip install torchmetrics

In [18]:
import torch
import neuralop
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm

# Import the dataset and processor
from neuralop.data.datasets.nonlinear_poisson import load_nonlinear_poisson_pt

# Import core neural operator components
from neuralop.layers.gno_block import GNOBlock
from neuralop.layers.integral_transform import IntegralTransform
#  Utilities for normalization, training loops, etc.
from neuralop.training import Trainer
from neuralop.utils import count_model_params, count_tensor_params


In [19]:
import matplotlib.pyplot as plt
import time
from torchmetrics import MeanSquaredError
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Downloading and checking if the data set is downloaded properly

In [6]:
!mkdir -p /content/neuralop_data
!wget -O /content/neuralop_data/nonlinear_poisson.obj https://zenodo.org/records/15001788/files/nonlinear_poisson.obj


--2025-11-28 21:07:55--  https://zenodo.org/records/15001788/files/nonlinear_poisson.obj
Resolving zenodo.org (zenodo.org)... 137.138.52.235, 188.185.48.75, 188.185.43.153, ...
Connecting to zenodo.org (zenodo.org)|137.138.52.235|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9608410281 (8.9G) [application/octet-stream]
Saving to: ‘/content/neuralop_data/nonlinear_poisson.obj’

/content/neuralop_d 100%[===================>]   8.95G  11.7MB/s    in 13m 4s  

2025-11-28 21:20:59 (11.7 MB/s) - ‘/content/neuralop_data/nonlinear_poisson.obj’ saved [9608410281/9608410281]



In [6]:
!ls -lh /content/neuralop_data/

total 9.0G
-rw-r--r-- 1 root root 9.0G Nov 28 21:04 nonlinear_poisson.obj


In [8]:
path = "/content/neuralop_data/nonlinear_poisson.obj"

# Load a small subset for testing (1 training + 1 testing instance)
train_loader, test_loader, data_processor = load_nonlinear_poisson_pt(
    data_path=path,
    n_train=1,
    n_test=1
)


File /content/neuralop_data/nonlinear_poisson.obj already exists. Skipping download.
Dictionary loaded successfully.
Loading 1 train instances...
Loading 1 test instances...


In [9]:
batch = next(iter(train_loader))
for k, v in batch.items():
    print(k, type(v), getattr(v, "shape", None))

x <class 'torch.Tensor'> torch.Size([1, 12000, 3])
input_geom <class 'torch.Tensor'> torch.Size([1, 12000, 2])
latent_queries <class 'torch.Tensor'> torch.Size([1, 48, 48, 2])
output_queries_domain <class 'torch.Tensor'> torch.Size([1, 6000, 2])
output_source_terms_domain <class 'torch.Tensor'> torch.Size([1, 6000])
y_domain <class 'torch.Tensor'> torch.Size([1, 6000])
coefs <class 'dict'> None
num_boundary <class 'torch.Tensor'> torch.Size([1])
out_sub_level <class 'torch.Tensor'> torch.Size([1])
y_bound <class 'torch.Tensor'> torch.Size([1, 1024])
output_queries_bound <class 'torch.Tensor'> torch.Size([1, 1024, 2])
output_source_terms_bound <class 'torch.Tensor'> torch.Size([1, 1024])


## MODEL

In [13]:
class GNO2DBlockEfficient(nn.Module):
    def __init__(self, in_channels, out_channels, coord_dim=2, radius=0.5, num_neighbors=128):
        super().__init__()
        self.coord_dim = coord_dim
        self.radius = radius
        self.num_neighbors = num_neighbors

        self.kernel_net = nn.Sequential(
            nn.Linear(coord_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

        self.feature_net = nn.Sequential(
            nn.Linear(in_channels, out_channels),
            nn.ReLU(),
            nn.Linear(out_channels, out_channels)
        )

    def forward(self, x, coords):
        """
        x:      [B, N, C_in]
        coords: [B, N, 2]
        """
        B, N, _ = x.shape
        device = x.device

        # Sample a subset of neighbors for each point
        idx = torch.randint(0, N, (B, N, self.num_neighbors), device=device)
        coord_neighbors = torch.gather(
            coords.unsqueeze(2).expand(-1, -1, N, -1), 2,
            idx.unsqueeze(-1).expand(-1, -1, -1, self.coord_dim)
        )  # [B, N, K, 2]

        x_neighbors = torch.gather(
            x.unsqueeze(2).expand(-1, -1, N, -1), 2,
            idx.unsqueeze(-1).expand(-1, -1, -1, x.shape[-1])
        )  # [B, N, K, C]

        rel = coords.unsqueeze(2) - coord_neighbors  # [B, N, K, 2]
        weights = self.kernel_net(rel).squeeze(-1)
        weights = F.softmax(-weights ** 2 / self.radius, dim=-1)

        agg = torch.sum(weights.unsqueeze(-1) * x_neighbors, dim=2)
        return self.feature_net(agg)


class SimpleGNOModelEfficient(nn.Module):
    def __init__(self, in_channels=3, hidden_channels=64, out_channels=1, n_layers=2):
        super().__init__()
        self.input_proj = nn.Linear(in_channels, hidden_channels)
        self.blocks = nn.ModuleList([
            GNO2DBlockEfficient(hidden_channels, hidden_channels) for _ in range(n_layers)
        ])
        self.output_proj = nn.Linear(hidden_channels, out_channels)

    def forward(self, x, coords):
        h = self.input_proj(x)
        for block in self.blocks:
            h = block(h, coords)
        return self.output_proj(h)

In [17]:
batch = next(iter(train_loader))
x = batch["x"]             # [1, 12000, 3]
geom = batch["input_geom"] # [1, 12000, 2]

model = SimpleGNOModelEfficient(in_channels=3, hidden_channels=64, out_channels=1, n_layers=2)
y_pred = model(x, geom)

print("Input:", x.shape)
print("Geom:", geom.shape)
print("Output:", y_pred.shape)

Input: torch.Size([1, 12000, 3])
Geom: torch.Size([1, 12000, 2])
Output: torch.Size([1, 12000, 1])


In [25]:
# ===============================================================
# LOSS, OPTIMIZER, SCHEDULER
# ===============================================================
criterion = nn.MSELoss()

def setup_training(model, lr=1e-3):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # Remove verbose=True for compatibility with older torch versions
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, factor=0.5, patience=5
    )
    return optimizer, scheduler




In [26]:
# ===============================================================
# TRAINING + VALIDATION FUNCTION
# ===============================================================
def run_epoch(model, loader, optimizer=None):
    """Run one epoch for training or validation."""
    is_train = optimizer is not None
    model.train(is_train)
    total_loss, count = 0.0, 0

    loop = tqdm(loader, desc="Train" if is_train else "Val", leave=False)
    for batch in loop:
        # Get data
        x = batch["x"].to(device)
        geom = batch["input_geom"].to(device)
        y_true = batch["y_domain"].to(device)  # ✅ correct supervision target

        # Forward
        y_pred = model(x, geom)
        loss = criterion(y_pred, y_true)

        if is_train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * x.size(0)
        count += x.size(0)
        loop.set_postfix(loss=loss.item())

    return total_loss / count


def train_and_validate(model, train_loader, val_loader, epochs=50, lr=1e-3):
    """Full training loop with validation + timing + plots."""
    model = model.to(device)
    optimizer, scheduler = setup_training(model, lr)

    train_losses, val_losses, epoch_times = [], [], []
    best_val = float("inf")

    print(f"🚀 Starting training for {epochs} epochs...\n")

    for epoch in range(1, epochs + 1):
        start_time = time.time()

        train_loss = run_epoch(model, train_loader, optimizer)
        val_loss = run_epoch(model, val_loader)

        scheduler.step(val_loss)
        elapsed = time.time() - start_time

        train_losses.append(train_loss)
        val_losses.append(val_loss)
        epoch_times.append(elapsed)

        print(f"Epoch {epoch:02d} | Train: {train_loss:.6f} | Val: {val_loss:.6f} | Time: {elapsed:.2f}s")

        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), "best_model.pt")

    print("\n✅ Training complete!")
    print(f"🏆 Best Validation Loss: {best_val:.6f}")

    # ---- Plot Loss and Time
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label="Train Loss")
    plt.plot(val_losses, label="Val Loss")
    plt.xlabel("Epoch")
    plt.ylabel("MSE Loss")
    plt.title("Training & Validation Loss")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(epoch_times, label="Time per Epoch (s)")
    plt.xlabel("Epoch")
    plt.ylabel("Seconds")
    plt.title("Epoch Duration")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    return {
        "train_losses": train_losses,
        "val_losses": val_losses,
        "times": epoch_times,
        "best_val": best_val
    }



In [27]:
# ===============================================================
# RUN TRAINING
# ===============================================================
model = SimpleGNOModelEfficient(in_channels=3, hidden_channels=64, out_channels=1, n_layers=2)
EPOCHS = 50
LR = 1e-3

results = train_and_validate(model, train_loader, test_loader, epochs=EPOCHS, lr=LR)


🚀 Starting training for 50 epochs...



Train:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/loss.py:634: UserWarning: Using a target size (torch.Size([1, 6000])) that is different to the input size (torch.Size([1, 12000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


OutOfMemoryError: CUDA out of memory. Tried to allocate 34.33 GiB. GPU 0 has a total capacity of 22.16 GiB of which 20.40 GiB is free. Process 208997 has 1.75 GiB memory in use. Of the allocated memory 1.45 GiB is allocated by PyTorch, and 81.06 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)